In [1]:
import pandas as pd

In [2]:
from collections import defaultdict

In [3]:
def compute_change(df):
    grouped_by_time = df.groupby('time')
    res = []
    sid_by_id = defaultdict(lambda: None) #记录每个人所对应的sid
    for key, group in grouped_by_time:
        index_by_sid = defaultdict(lambda: None) #记录每个sensor在sids里面的index
        time = int(key)
        values = group[['sid', 'id']].values
        sids = [] #记录当前时间点所有发生变化的sensor区域
        index = 0
        for item in values:
            sid_key = str(item[0])
            id_key = str(item[1])

            #计算增加的人
            if not index_by_sid[sid_key]:
                index_by_sid[sid_key] = index
                index += 1
                sids.append([int(sid_key), 1])
            else:
                sids[index_by_sid[sid_key]][1] += 1

            #计算减少的人
            if sid_by_id[id_key]:
                #如果在会场里面
                sid_key_old = sid_by_id[id_key]
                if not index_by_sid[sid_key_old]:
                    #如果这个传感器没有出现，就增加
                    index_by_sid[sid_key_old] = index
                    index += 1
                    sids.append([int(sid_key_old), -1])
                else:
                    #如果这个传感器出现了，就修改
                    sids[index_by_sid[sid_key_old]][1] -= 1
                    
            #更新位置
            sid_by_id[id_key] = int(sid_key)

        res.append({
            't':time,
            's':sids
        })
    return res

In [4]:
def save_to_csv(des, res):
    with open(des, 'a') as f:
        for item in res:
            words = []
            words.append(str(item['time']))
            for sensor in item['sids']:
                words.append(str(sensor['sid']))
                words.append(str(sensor['change']))
            f.write(','.join(words) + '\n')

In [5]:
import json
def save_to_json(data, des):
    with open(des, 'w') as f:
        json.dump(data, f, ensure_ascii=False)

In [6]:
def transform(src, des):
    df = pd.read_csv(src)
    data = compute_change(df)
    save_to_json(data, des)

In [7]:
# transform("../data/logs/day1.csv", "../data/time/time_day1.json")

In [8]:
transform("../data/logs/day2.csv", "../data/time/time_day2.json")

In [9]:
# transform("day3.csv", "day3_time_sensor_change.csv")